# Import the necessary libraries

In [1]:
import pandas as pd
import sys
import os
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import numpy as np  # Make sure numpy is imported

# Configure logging
logging.basicConfig(filename='../logs/predict_store_sales_with_deep_learning.log',
                    level=logging.INFO,
                    format='%(asctime)s:%(levelname)s:%(message)s')
  
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../scripts')))
from load_data import Load_Data
# from sklearn.compose import ColumnTransformer
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from sklearn.pipeline import Pipeline
# from sklearn.metrics import mean_squared_error, mean_absolute_error


# Load data set

In [2]:
logging.info("Load cleaned store data")

# Create an instance of CSVReader
df = Load_Data('../data/combined_cleaned_data.csv')

# Load the data
df.load_data()

# Get the loaded data
df = df.get_data()


Data successfully loaded from ../data/combined_cleaned_data.csv
